# Creating Russian Classifier

In [1]:
import pandas as pd
from telethon import TelegramClient
from datetime import datetime, timedelta, timezone

from app.api import *

from sklearn.metrics import classification_report

## Get Dataset of Russian Messages

In [3]:
async def main(chat_name):
    global api_id
    global api_hash
    messages = []
    async with TelegramClient(api_id=api_id, api_hash=api_hash, session='this') as client:
            async for message in client.iter_messages(chat_name, offset_date=datetime.now(tz=timezone.utc) - timedelta(days = 365), reverse=True):
                messages.append(message)
    return messages

In [4]:
res = await main('astrapress')

Signed in successfully as Jank Mood; remember to not break the ToS or you will risk an account ban!


In [5]:
len(res)

27330

In [6]:
texts = [message.text for message in res]

In [7]:
texts[0]

'**Сегодня в Грозном прошел митинг против сожжения Корана**\n\nКак пишет местное издание «Грозный-Информ», жители республики возмутились акцией Никиты Журавеля в Волгограде, который сжег Коран напротив мечети. \n\nПришедшие держали в руках плакаты __«Я люблю Коран», «Сатанистам нет места на Земле», «Нет культуры и религии у того, кто посягает на святое__». \n\nРанее Бастрыкин передал дело 19-летнего Никиты Журавеля чеченским силовикам. СК объяснил передачу дела в Чечню тем, что жители республики якобы обращались с просьбой признать их потерпевшими.'

In [8]:
texts_df = pd.DataFrame(texts)

In [9]:
texts_df.head()

,0
0,**Сегодня в Грозном прошел митинг против сожже...
1,
2,
3,**На **[**митинг**](https://t.me/astrapress/27...
4,В Белгороде сообщили о звуках взрыва


In [10]:
texts_df.to_csv('ru_messages.csv', index=False)

In [9]:
import pandas as pd

In [10]:
texts_df = pd.read_csv('ru_messages.csv')

In [7]:
texts_df.head()

,0
0,**Сегодня в Грозном прошел митинг против сожже...
1,NaN
2,NaN
3,**На **[**митинг**](https://t.me/astrapress/27...
4,В Белгороде сообщили о звуках взрыва


In [7]:
texts_df = texts_df.dropna()

In [8]:
texts_df = texts_df.drop_duplicates()

In [20]:
texts_df.to_csv('ru_messages.csv', index=False)

In [12]:
texts_df['0'][0]

'**Сегодня в Грозном прошел митинг против сожжения Корана**\n\nКак пишет местное издание «Грозный-Информ», жители республики возмутились акцией Никиты Журавеля в Волгограде, который сжег Коран напротив мечети. \n\nПришедшие держали в руках плакаты __«Я люблю Коран», «Сатанистам нет места на Земле», «Нет культуры и религии у того, кто посягает на святое__». \n\nРанее Бастрыкин передал дело 19-летнего Никиты Журавеля чеченским силовикам. СК объяснил передачу дела в Чечню тем, что жители республики якобы обращались с просьбой признать их потерпевшими.'

## Translate Russian Messages to English

In [1]:
from transformers import pipeline

2024-05-22 17:35:17.212322: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-22 17:35:17.212373: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-22 17:35:17.212414: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-22 17:35:17.221945: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/j/.local/lib/python3.10/site-packages/transf

In [2]:
translator = pipeline("translation_ru_to_en", model="Helsinki-NLP/opus-mt-ru-en", device=0)

In [3]:
import pandas as pd
texts_df = pd.read_csv('ru_messages.csv')

In [4]:
def translate(text):
    try:
        return translator(text[:400])
    except:
        return 'error'

In [5]:
texts_df['translation'] = [translate(text) for text in texts_df['0']]

/home/j/.local/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [9]:
successes = texts_df[texts_df['translation'] != 'error']
len(successes)

15965

In [10]:
successes.to_csv('ru_en_messages.csv')

## Annotate With English Classifier

In [1]:
import pandas as pd

In [2]:
copy = pd.read_csv('ru_en_messages.csv')

In [3]:
from classifier import XGBClassifier

In [4]:
classifier = XGBClassifier('/home/j/Documents/Projects/social-media-combat-detection/combatmessagesua/telegram/models/xgb_classifier','/home/j/Documents/Projects/social-media-combat-detection/combatmessagesua/telegram/models/tfid-vectorizer.pickle')

Classifier loaded
Vectorizer loaded


In [9]:
copy['translation'][0]

'[{\'translation_text\': \'** "The Head" of the Hersonian region, Vladimir Saldo, posted a video of the Russian military in Krynka** "The lower bow, heroes, from all the residents of Hersonshchina and my personal great gratitude!" he signed. He said that in the video, the Marines of the 810th Guard Brigade and the paratroopers called the Ukrainian side (https://t.me/astrapress/48904) "manipulation and falsification" of Schoygou\\\'s statement about the taking of the Krunks.\'}]'

In [5]:
import xgboost

In [6]:
def pred(message):
    vectorized = xgboost.DMatrix(classifier.vectorizer.transform([message]))
    if classifier.classifier.predict(vectorized) >= 0.5:
        return 1
    else:
        return 0

In [7]:
copy['annotation'] = [pred(text) for text in copy['translation']]

In [8]:
copy.head()

,Unnamed: 0,0,translation,annotation
0,0,**Сегодня в Грозном прошел митинг против сожже...,"[{'translation_text': '**Today, in Grozny, the...",0
1,1,**На **[**митинг**](https://t.me/astrapress/27...,[{'translation_text': '** **[**miting**] (http...,0
2,2,В Белгороде сообщили о звуках взрыва,"[{'translation_text': 'In Belgorod, the sounds...",1
3,3,"Над Белгородом после взрыва поднимается дым, п...","[{'translation_text': ""There's smoke rising ov...",1
4,4,«Белгород-Молния»: взрыв прогремел в районе ж/...,[{'translation_text': 'Belgorod-Molniya: explo...,0


In [9]:
copy.to_csv('annotated_ru_en_messages.csv', index=False)

## Train New Classifier

In [1]:
import pandas as pd
df = pd.read_csv('annotated_ru_en_messages.csv')
df.head()

,Unnamed: 0,0,translation,annotation
0,0,**Сегодня в Грозном прошел митинг против сожже...,"[{'translation_text': '**Today, in Grozny, the...",0
1,1,**На **[**митинг**](https://t.me/astrapress/27...,[{'translation_text': '** **[**miting**] (http...,0
2,2,В Белгороде сообщили о звуках взрыва,"[{'translation_text': 'In Belgorod, the sounds...",1
3,3,"Над Белгородом после взрыва поднимается дым, п...","[{'translation_text': ""There's smoke rising ov...",1
4,4,«Белгород-Молния»: взрыв прогремел в районе ж/...,[{'translation_text': 'Belgorod-Molniya: explo...,0


### Balance Dataset

In [6]:
ones = df[df['annotation']==1]
len(ones)

2738

In [7]:
zeros = df[df['annotation']==0]
len(zeros)

13227

In [8]:
zero_sample = zeros.sample(n=len(ones))

In [13]:
ones = ones.sample(frac=1)
zero_sample = zero_sample.sample(frac=1)

In [20]:
train_df = pd.concat([ones[:int(.9*len(ones))], zero_sample[:int(.9*len(ones))]]).sample(frac=1)
test_df = pd.concat([ones[int(.9*len(ones)):], zero_sample[int(.9*len(ones)):]]).sample(frac=1)

In [21]:
train_df.head()

,Unnamed: 0,0,translation,annotation
6252,6252,**Актриса Яна Поплавская выступила за лишение ...,[{'translation_text': 'If you\'re not just an ...,0
10698,10698,**Роскомнадзор хочет узнать геолокацию всех IP...,[{'translation_text': '**Roskomnadzor wants to...,0
10880,10880,"4 человека погибли, 42 ранены в результате обс...",[{'translation_text': 'Four people were killed...,1
15691,15691,**Появились спутниковые снимки аэродрома Бельб...,[{'translation_text': '** Satellite images of ...,1
10793,10793,**Порт Усть-Луга в Ленинградской области атако...,[{'translation_text': '**The port of Ust-Luga ...,1


In [22]:
test_df.head()

,Unnamed: 0,0,translation,annotation
14639,14639,В России [заблокировали](https://t.me/roskomsv...,[{'translation_text': 'In Russia (https://t.me...,0
5515,5515,**Один человек ранен в результате ночной трёхч...,[{'translation_text': '** One person was injur...,1
9514,9514,"Ночью о взрывах сообщили жители Киева, Черкасс...","[{'translation_text': 'At night, the inhabitan...",1
9709,9709,**Надпись «Где Навальный?» появилась на здании...,"[{'translation_text': '** The inscription ""Whe...",0
15851,15851,В Луганске сообщают о взрыве\n\nUpd: По предва...,"[{'translation_text': 'In Luhansk, there are r...",1


### Train New Tokenizer

In [28]:
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier

from sklearn.metrics import classification_report

from sklearn.feature_extraction.text import TfidfVectorizer

In [25]:
vectorizer = TfidfVectorizer(sublinear_tf=True)

freq_matrix_train = vectorizer.fit_transform(train_df['0'])
freq_matrix_test = vectorizer.transform(test_df['0'])

In [32]:
import pickle
with open('/home/j/Documents/Projects/social-media-combat-detection/models/ru-tfidf-vectorizer.pickle', 'wb') as handle:
    pickle.dump(vectorizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

### Tune Hyperparameters

In [29]:
search = RandomizedSearchCV(XGBClassifier(), {'n_estimators': [int(x) for x in range(200,2000,200)],
        'min_child_weight': range(1,6,2),
        'gamma': [i/10.0 for i in range(0,5)],
        'subsample': [i/10.0 for i in range(6,10)],
        'colsample_bytree': [i/10.0 for i in range(6,10)],
        'max_depth': [3, 6, 9],
        'learning_rate': [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3]
        })


In [30]:
search.fit(freq_matrix_train, train_df['annotation'].astype(int))

best = search.best_params_

In [31]:
print(best)

{'subsample': 0.9, 'n_estimators': 200, 'min_child_weight': 1, 'max_depth': 9, 'learning_rate': 0.1, 'gamma': 0.2, 'colsample_bytree': 0.7}


In [33]:
classifier = XGBClassifier(subsample= best['subsample'], 
                           n_estimators= best['n_estimators'], 
                           min_child_weight= best['min_child_weight'], 
                           max_depth= best['max_depth'], 
                           learning_rate= best['learning_rate'], 
                           gamma= best['gamma'], 
                           colsample_bytree= best['colsample_bytree'])

classifier.fit(freq_matrix_train, train_df['annotation'].astype(int))

y_pred = classifier.predict(freq_matrix_test)

print(classification_report(test_df['annotation'].astype(int),y_pred))

              precision    recall  f1-score   support

           0       0.91      0.85      0.88       274
           1       0.86      0.92      0.89       274

    accuracy                           0.89       548
   macro avg       0.89      0.89      0.89       548
weighted avg       0.89      0.89      0.89       548



In [35]:
classifier.save_model('/home/j/Documents/Projects/social-media-combat-detection/models/ru-xgb-classifier')

/home/j/.local/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [17:46:26] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)
